# Домашнее задание № 3. Языковые модели

## Задание 1 (8 баллов).

В семинаре для генерации мы использовали предположение маркова и считали, что слово зависит только от 1 предыдущего слова. Но ничто нам не мешает попробовать увеличить размер окна и учитывать два или даже три прошлых слова. Для них мы еще сможем собрать достаточно статистик и, логично предположить, что качество сгенерированного текста должно вырасти.

Попробуйте сделать языковую модель, которая будет учитывать два предыдущих слова при генерации текста.
Сгенерируйте несколько текстов (3-5) и расчитайте перплексию получившейся модели. 
Можно использовать данные из семинара или любые другие (можно брать только часть текста, если считается слишком долго). Перплексию рассчитывайте на 10-50 отложенных предложениях (они не должны использоваться при сборе статистик).


Подсказки:  
    - нужно будет добавить еще один тэг \<start>  
    - можете использовать тот же подход с матрицей вероятностей, но по строкам хронить биграмы, а по колонкам униграммы 
    - тексты должны быть очень похожи на нормальные (если у вас получается рандомная каша, вы что-то делаете не так)
    - у вас будут словари с индексами биграммов и униграммов, не перепутайте их при переводе индекса в слово - словарь биграммов будет больше словаря униграммов и все индексы из униграммного словаря будут формально подходить для словаря биграммов (не будет ошибки при id2bigram[unigram_id]), но маппинг при этом будет совершенно неправильным 

In [3]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
import matplotlib.pyplot as plt
import zipfile
from collections import Counter
from nltk.tokenize import sent_tokenize
from scipy.sparse import lil_matrix, csr_matrix, csc_matrix

with zipfile.ZipFile('habr_texts.txt.zip', 'r') as zip_file:
    with zip_file.open('habr_texts.txt', 'r') as txt_file:
        habr = txt_file.read().decode('utf-8')

news = open('lenta.txt', 'r', encoding='utf-8').read()
habr = habr[:12_000_000]

def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text

norm_habr = normalize(habr)
norm_news = normalize(news)

vocab_habr = Counter(norm_habr)
vocab_news = Counter(norm_news)

probas_habr = Counter({word:c/len(norm_habr) for word, c in vocab_habr.items()})
probas_news = Counter({word:c/len(norm_news) for word, c in vocab_news.items()})

def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

all_sentences_habr = [['<start>', '<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(habr[:5000000])]
all_sentences_news = [['<start>', '<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(news[:5000000])]

train_size_habr = int(0.9 * len(all_sentences_habr))
train_sentences_habr = all_sentences_habr[:train_size_habr]
test_sentences_habr = all_sentences_habr[train_size_habr:train_size_habr + 30]


train_size_news = int(0.9 * len(all_sentences_news))
train_sentences_news = all_sentences_news[:train_size_news]
test_sentences_news = all_sentences_news[train_size_news:train_size_news + 30]

unigrams_habr = Counter()
bigrams_habr = Counter()
trigrams_habr = Counter()

for sentence in train_sentences_habr:
    unigrams_habr.update(sentence)
    bigrams_habr.update(ngrammer(sentence, n=2))
    trigrams_habr.update(ngrammer(sentence, n=3))


unigrams_news = Counter()
bigrams_news = Counter()
trigrams_news = Counter()

for sentence in train_sentences_news:
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer(sentence, n=2))
    trigrams_news.update(ngrammer(sentence, n=3))

print(f"Хабр - Униграмм: {len(unigrams_habr)}, Биграмм: {len(bigrams_habr)}, Триграмм: {len(trigrams_habr)}")
print(f"Лента - Униграмм: {len(unigrams_news)}, Биграмм: {len(bigrams_news)}, Триграмм: {len(trigrams_news)}")

Хабр - Униграмм: 74857, Биграмм: 396196, Триграмм: 544129
Лента - Униграмм: 68186, Биграмм: 349724, Триграмм: 501998


In [4]:
bigram_vocab_habr = set()
for trigram in trigrams_habr:
    words = trigram.split()
    if len(words) == 3:
        bigram = ' '.join(words[:2])
        bigram_vocab_habr.add(bigram)

bigram_vocab_news = set()
for trigram in trigrams_news:
    words = trigram.split()
    if len(words) == 3:
        bigram = ' '.join(words[:2])
        bigram_vocab_news.add(bigram)

matrix_trigram_habr = lil_matrix((len(bigram_vocab_habr), len(unigrams_habr)))
matrix_trigram_news = lil_matrix((len(bigram_vocab_news), len(unigrams_news)))

id2word_habr = list(unigrams_habr)
word2id_habr = {word:i for i, word in enumerate(id2word_habr)}

id2word_news = list(unigrams_news)
word2id_news = {word:i for i, word in enumerate(id2word_news)}

id2bigram_habr = list(bigram_vocab_habr)
bigram2id_habr = {bigram:i for i, bigram in enumerate(id2bigram_habr)}

id2bigram_news = list(bigram_vocab_news)
bigram2id_news = {bigram:i for i, bigram in enumerate(id2bigram_news)}

for trigram in trigrams_habr:
    words = trigram.split()
    if len(words) == 3:
        word1, word2, word3 = words
        bigram = ' '.join([word1, word2])
        
        if bigram in bigrams_habr and bigrams_habr[bigram] > 0:
            prob = trigrams_habr[trigram] / bigrams_habr[bigram]
            if bigram in bigram2id_habr and word3 in word2id_habr:
                matrix_trigram_habr[bigram2id_habr[bigram], word2id_habr[word3]] = prob

for trigram in trigrams_news:
    words = trigram.split()
    if len(words) == 3:
        word1, word2, word3 = words
        bigram = ' '.join([word1, word2])
        
        if bigram in bigrams_news and bigrams_news[bigram] > 0:
            prob = trigrams_news[trigram] / bigrams_news[bigram]
            if bigram in bigram2id_news and word3 in word2id_news:
                matrix_trigram_news[bigram2id_news[bigram], word2id_news[word3]] = prob

matrix_habr = csc_matrix(matrix_trigram_habr)
matrix_news = csc_matrix(matrix_trigram_news)

print(f"Матрица хабра: {matrix_habr.shape}")
print(f"Матрица ленты: {matrix_news.shape}")

Матрица хабра: (382458, 74857)
Матрица ленты: (338518, 68186)


In [5]:
def generate_trigram(matrix_trigram, id2bigram, bigram2id, id2word, word2id, n=100, start_bigram='<start> <start>'):
    text = []
    
    current_bigram = start_bigram
    words = current_bigram.split()
    text.extend(words)
    
    for i in range(n):
        if current_bigram not in bigram2id:
            if len(words) >= 1:
                last_word = words[-1]
                text.append('<end>')
                break
            else:
                text.append('<end>')
                break
        
        bigram_idx = bigram2id[current_bigram]
        
        prob_row = matrix_trigram[bigram_idx].toarray()[0]
        
        if prob_row.sum() == 0:
            text.append('<end>')
            break
        
        prob_row = prob_row / prob_row.sum()
        
        chosen_word_idx = np.random.choice(len(prob_row), p=prob_row)
        chosen_word = id2word[chosen_word_idx]
        
        text.append(chosen_word)
        
        if chosen_word == '<end>':
            break
        
        words = current_bigram.split()
        current_bigram = f"{words[1]} {chosen_word}"
    
    return ' '.join(text)

In [6]:
for i in range(5):
    text = generate_trigram(
        matrix_habr, 
        id2bigram_habr, 
        bigram2id_habr, 
        id2word_habr, 
        word2id_habr,
        n=20
    )
    print(f"{i+1}. {text}")


1. <start> <start> кроме того необходимо будет быстро исправить <end>
2. <start> <start> таким образом я получаю веб-страницы с других моделях <end>
3. <start> <start> уверен что по атрибутам label … указывает использовать ли markdown теги в тексте прошу направлять в лс <end>
4. <start> <start> каждый чиллер с насосом трубопроводом и теплообменником образует независимый холодильный узел <end>
5. <start> <start> база исполнителей всё время <end>


In [7]:
for i in range(5):
    text = generate_trigram(
        matrix_news, 
        id2bigram_news, 
        bigram2id_news, 
        id2word_news, 
        word2id_news,
        n=20
    )
    print(f"{i+1}. {text}")

1. <start> <start> семинар призван определить правила развития застройки земель в россии 6,3 миллиона семей стоят в очереди на жилье <end>
2. <start> <start> жалобы фермера в полицию не принесли никаких результатов подтверждающих заявления представителей швейцарских властей <end>
3. <start> <start> цик сомневается правильно ли они а также планы развития этих отношений <end>
4. <start> <start> за истекшие сутки зарегистрировано 18 дтп в результате два человека погибли и более уместных на выставке демонстрируются последние достижения в
5. <start> <start> интересно при этом энергосистема лишь на тех кто напрямую связывает это преступление с борьбой вокруг решения о расширении состава ес


In [8]:
def compute_joint_proba_trigram_markov(text, word_counts, bigram_counts, trigram_counts):
    prob = 0
    tokens = normalize(text)
    
    sequence = ['<start>', '<start>'] + tokens + ['<end>']
    
    for i in range(2, len(sequence)):
        word1, word2, word3 = sequence[i-2], sequence[i-1], sequence[i]
        trigram = f"{word1} {word2} {word3}"
        bigram = f"{word1} {word2}"
        
        if bigram in bigram_counts and bigram_counts[bigram] > 0 and trigram in trigram_counts:
            prob += np.log(trigram_counts[trigram] / bigram_counts[bigram])
        else:
            prob += np.log(2e-5)
    
    return np.exp(prob)

In [9]:
def perplexity(logp, N):
    return np.exp((-1/N) * logp)

def compute_joint_proba_trigram(text, word_counts, bigram_counts, trigram_counts):

    prob = 0
    tokens = normalize(text)
    
    sequence = ['<start>', '<start>'] + tokens + ['<end>']
    
    for i in range(2, len(sequence)):
        word1, word2, word3 = sequence[i-2], sequence[i-1], sequence[i]
        trigram = f"{word1} {word2} {word3}"
        bigram = f"{word1} {word2}"
        
        if bigram in bigram_counts and bigram_counts[bigram] > 0 and trigram in trigram_counts:
            prob += np.log(trigram_counts[trigram] / bigram_counts[bigram])
        else:
            prob += np.log(2e-5)
    
    return prob, len(tokens)

def calculate_perplexity_on_test_set(test_sentences, word_counts, bigram_counts, trigram_counts):

    total_log_prob = 0
    total_words = 0
    
    for sentence in test_sentences:
        text = ' '.join([word for word in sentence if word not in ['<start>', '<end>']])
        log_prob, num_words = compute_joint_proba_trigram(text, word_counts, bigram_counts, trigram_counts)
        total_log_prob += log_prob
        total_words += num_words
    
    return perplexity(total_log_prob, total_words)

perp_habr = calculate_perplexity_on_test_set(test_sentences_habr, unigrams_habr, bigrams_habr, trigrams_habr)
perp_news = calculate_perplexity_on_test_set(test_sentences_news, unigrams_news, bigrams_news, trigrams_news)

print(f"Перплексия Хабра: {perp_habr:.2f}")
print(f"Перплексия Ленты: {perp_news:.2f}")

Перплексия Хабра: 20866.24
Перплексия Ленты: 16479.54


## Задание № 2 (2 балла). 

Измените функцию generate_with_beam_search так, чтобы она работала с моделью, которая учитывает два предыдущих слова. 
Сравните получаемый результат с первым заданием. 
Также попробуйте начинать генерацию не с нуля (подавая \<start> \<start>), а с какого-то промпта. Но помните, что учитываться будут только два последних слова, так что не делайте длинные промпты.

In [10]:
class Beam:
    def __init__(self, sequence: list, score: float):
        self.sequence: list = sequence
        self.score: float = score


def generate_with_beam_search_trigram(matrix_trigram, id2bigram, bigram2id, id2word, word2id, n=100, max_beams=5, start_bigram='<start> <start>'):

    initial_sequence = start_bigram.split()
    initial_node = Beam(sequence=initial_sequence, score=0.0)
    beams = [initial_node]
    
    for i in range(n):
        new_beams = []
        
        for beam in beams:
            if beam.sequence and beam.sequence[-1] == '<end>':
                new_beams.append(beam)
                continue
            
            if len(beam.sequence) >= 2:
                current_bigram = ' '.join(beam.sequence[-2:])
            else:
                if len(beam.sequence) == 1:
                    current_bigram = f"<start> {beam.sequence[-1]}"
                else:
                    current_bigram = "<start> <start>"
            
            if current_bigram not in bigram2id:
                new_beams.append(beam)
                continue
            
            bigram_idx = bigram2id[current_bigram]
            probas = matrix_trigram[bigram_idx].toarray()[0]
            
            valid_indices = np.where(probas > 0)[0]
            if len(valid_indices) == 0:
                new_beams.append(beam)
                continue
            
            top_idxs = valid_indices[probas[valid_indices].argsort()[::-1][:max_beams]]
            
            for top_id in top_idxs:
                next_word = id2word[top_id]
                
                new_sequence = beam.sequence + [next_word]
                
                word_prob = probas[top_id]
                new_score = (beam.score * len(beam.sequence) + np.log(word_prob + 1e-10)) / len(new_sequence)
                
                new_beam = Beam(sequence=new_sequence, score=new_score)
                new_beams.append(new_beam)
        
        if new_beams:
            beams = sorted(new_beams, key=lambda x: x.score, reverse=True)[:max_beams]
        else:
            break
    
    sorted_beams = sorted(beams, key=lambda x: x.score, reverse=True)
    return [" ".join(beam.sequence) for beam in sorted_beams]

In [11]:
results_news = generate_with_beam_search_trigram(
    matrix_news, id2bigram_news, bigram2id_news, id2word_news, word2id_news,
    n=15, max_beams=5
)
for i, text in enumerate(results_news[:3]):
    print(f"{i+1}. {text}")

1. <start> <start> как сообщает риа новости со ссылкой на источники в правоохранительных органах города интерфаксу сообщили в
2. <start> <start> как сообщает риа новости <end>
3. <start> <start> как сообщает риа новости со ссылкой на источники в правоохранительных органах города интерфаксу сообщили что


In [12]:
results_habr = generate_with_beam_search_trigram(
    matrix_habr, id2bigram_habr, bigram2id_habr, id2word_habr, word2id_habr,
    n=15, max_beams=5
)
for i, text in enumerate(results_habr[:3]):  # Показываем топ-3
    print(f"{i+1}. {text}")

1. <start> <start> если у вас есть таблица состоящая из выпускников и студентов междисциплинарных программ вандербильта и факультета
2. <start> <start> если у вас есть таблица состоящая из выпускников и студентов междисциплинарных программ вандербильта и медицинской
3. <start> <start> если у вас есть опыт по внедрению оценке и использованию систем геймификации в профильных сообществах


In [14]:
def generate_with_prompt_trigram(matrix_trigram, id2bigram, bigram2id, id2word, word2id, prompt_text, n=50, max_beams=5):

    prompt_tokens = normalize(prompt_text)
    
    if len(prompt_tokens) >= 2:
        start_bigram = ' '.join(prompt_tokens[-2:])
    elif len(prompt_tokens) == 1:
        start_bigram = f"<start> {prompt_tokens[0]}"
    else:
        start_bigram = "<start> <start>"
    
    return generate_with_beam_search_trigram(
        matrix_trigram, id2bigram, bigram2id, id2word, word2id,
        n=n, max_beams=max_beams, start_bigram=start_bigram
    )

In [26]:
prompts = [
    "сегодня",
    "ежедневно в",
    "пришлось",
    "вероятно",
    "россия"
]

In [27]:
for prompt in prompts:
    print(f"\nПромпт: '{prompt}'")
    results = generate_with_prompt_trigram(
        matrix_news, id2bigram_news, bigram2id_news, id2word_news, word2id_news,
        prompt, n=10, max_beams=3
    )
    for i, text in enumerate(results[:3]):
        print(f"  {i+1}. {text}")


Промпт: 'сегодня'
  1. <start> сегодня же скупщина черногории приняла закон о внесении изменений и дополнений
  2. <start> сегодня же скупщина черногории приняла закон о гарантировании вкладов граждан в
  3. <start> сегодня же скупщина черногории приняла закон о гарантировании вкладов граждан вбанках

Промпт: 'ежедневно в'
  1. ежедневно в освобожденные районы возвращаются по 600-800 человек <end>
  2. ежедневно в освобожденные районы чечни <end>
  3. ежедневно в освобожденные села <end>

Промпт: 'пришлось'
  1. <start> пришлось отменить сотни авиарейсов в города флориды и джорджии спешат уехать
  2. <start> пришлось отменить сотни авиарейсов в города флориды и джорджии а также
  3. <start> пришлось отменить сотни авиарейсов непростая ситуация сложилась на рынке операционных систем

Промпт: 'вероятно'
  1. <start> вероятно окончательное число погибших достигло 36 человек в том числе и
  2. <start> вероятно окончательное число погибших достигло 36 человек в том числе в
  3. <start> вер

In [29]:
for prompt in prompts:
    print(f"\nПромпт: '{prompt}'")
    results = generate_with_prompt_trigram(matrix_habr, id2bigram_habr, bigram2id_habr, id2word_habr, word2id_habr, prompt, n=15, max_beams=5)
    for i, text in enumerate(results[:3]):
        print(f"  {i+1}. {text}")


Промпт: 'сегодня'
  1. <start> сегодня это 1 nbsp 190 долларов <end>
  2. <start> сегодня это 1 nbsp 254 долларов <end>
  3. <start> сегодня это 1 nbsp 000 долларов <end>

Промпт: 'ежедневно в'
  1. ежедневно в мире telegram-банк онлайн-образование пятизвездочный отель на берегу теплого моря и поиск друзей hola приглашает full-stack
  2. ежедневно в мире telegram-банк онлайн-образование пятизвездочный отель на берегу теплого моря и поиск гениев « гении равномерно
  3. ежедневно в мире telegram-банк онлайн-образование пятизвездочный отель на берегу теплого моря и поиск « живых » ipv

Промпт: 'пришлось'
  1. <start> пришлось критически пересмотреть всё имеющееся научное знание по этой причине власти великобритании предлагают выделить £ 1,9
  2. <start> пришлось реализовывать обертку которая разбирает вывод утилиты в stdout на ошибки в pri потоке приведут к
  3. <start> пришлось критически пересмотреть всё имеющееся научное знание по этой ссылке <end>

Промпт: 'вероятно'
  1. <start> вер

По сравнению с генерациями в первом задании beam_search приводит к однотипным предложениям, но более связным. Возможности beam search ограничены, в моём случае с длинными промптами такая генерация не работает